In [1]:
import pandas as pd
df2 = pd.read_csv("London Stock Exchange Stock Price History.csv")
df = df2.copy()
df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,09/08/2016,"2,736.0","2,702.0","2,754.0","2,702.0",656.21K,-0.29%
1,09/07/2016,"2,744.0","2,755.0","2,766.0","2,730.0",262.97K,0.18%
2,09/06/2016,"2,739.0","2,739.0","2,748.0","2,725.0",438.83K,-0.22%
3,09/05/2016,"2,745.0","2,801.0","2,801.0","2,745.0",158.52K,-0.80%
4,09/02/2016,"2,767.0","2,770.0","2,782.0","2,734.0",676.31K,0.62%


In [2]:
df = df.drop(columns=['Vol.'])

In [3]:
df.describe().T

,count,unique,top,freq
Date,1794,1794,09/08/2016,1
Price,1794,1261,"2,462.0",7
Open,1794,1360,8.0,6
High,1794,1363,8.1,7
Low,1794,1373,"2,564.0",5
Change %,1794,626,0.00%,33


In [5]:
df.isnull().sum()

Date        0
Price       0
Open        0
High        0
Low         0
Change %    0
dtype: int64

In [ ]:
# "Hac." sütununu dönüştürmek için fonksiyon
def convert_volume(value):
    value = value.replace(',', '.')  # Virgülü noktaya çevir
    if value.endswith('M'):
        return float(value[:-1]) * 1_000_000  # "M" ise milyonla çarp
    elif value.endswith('K'):
        return float(value[:-1]) * 1_000      # "K" ise binle çarp
    else:
        return float(value)                    # Kısaltma yoksa direkt float

# Fonksiyonu "Hac." sütununa uygulama
df['Hac.'] = df['Hac.'].apply(convert_volume)

In [6]:
df.head()

,Date,Price,Open,High,Low,Change %
0,09/08/2016,"2,736.0","2,702.0","2,754.0","2,702.0",-0.29%
1,09/07/2016,"2,744.0","2,755.0","2,766.0","2,730.0",0.18%
2,09/06/2016,"2,739.0","2,739.0","2,748.0","2,725.0",-0.22%
3,09/05/2016,"2,745.0","2,801.0","2,801.0","2,745.0",-0.80%
4,09/02/2016,"2,767.0","2,770.0","2,782.0","2,734.0",0.62%


In [12]:
# "Tarih" sütununu datetime formatına çevirme
#df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
df['Date'] = pd.to_datetime(df['Date'],format='%m/%d/%Y')
df.head()

,Date,Price,Open,High,Low,Change %
0,2016-09-08,"2,736.0","2,702.0","2,754.0","2,702.0",-0.29%
1,2016-09-07,"2,744.0","2,755.0","2,766.0","2,730.0",0.18%
2,2016-09-06,"2,739.0","2,739.0","2,748.0","2,725.0",-0.22%
3,2016-09-05,"2,745.0","2,801.0","2,801.0","2,745.0",-0.80%
4,2016-09-02,"2,767.0","2,770.0","2,782.0","2,734.0",0.62%


In [13]:
# Eksik verileri kontrol etme
print(df.isnull().sum())

# Eksik verileri bir önceki değerle doldurma
#df = df.fillna(method='ffill')

Date        0
Price       0
Open        0
High        0
Low         0
Change %    0
dtype: int64


In [14]:
# İlk birkaç satırı görüntüleme
print(df.head())

# Sütun tiplerini kontrol etme
print(df.dtypes)

        Date    Price     Open     High      Low Change %
0 2016-09-08  2,736.0  2,702.0  2,754.0  2,702.0   -0.29%
1 2016-09-07  2,744.0  2,755.0  2,766.0  2,730.0    0.18%
2 2016-09-06  2,739.0  2,739.0  2,748.0  2,725.0   -0.22%
3 2016-09-05  2,745.0  2,801.0  2,801.0  2,745.0   -0.80%
4 2016-09-02  2,767.0  2,770.0  2,782.0  2,734.0    0.62%
Date        datetime64[ns]
Price               object
Open                object
High                object
Low                 object
Change %            object
dtype: object


In [16]:
# Tarih sütununun zaten datetime64[ns] formatında olduğunu varsayıyoruz
# Fiyat sütunlarını sayısal formata dönüştürme
#for col in ['Price', 'Open', 'High', 'Low']:
    #df[col] = df[col].str.replace(',', '.').astype(float)

for col in ['Price', 'Open', 'High', 'Low']:
    # Binlik ayırıcı noktaları kaldır
    df[col] = df[col].str.replace('.', '', regex=False)  # Önce tüm noktaları kaldır
    # Daha sonra ondalık ayırıcının da noktadan farklı olduğunu varsayarsak değiştirebilirsin
    df[col] = df[col].str.replace(',', '.', regex=False)  # Virgülü noktaya çevir
    df[col] = df[col].astype(float)

# Dönüşüm sonrası sütun tiplerini kontrol etme
print(df.dtypes)

Date        datetime64[ns]
Price              float64
Open               float64
High               float64
Low                float64
Change %            object
dtype: object


In [17]:
# 10 günlük SMA ekleme
df['SMA_10'] = df['Price'].rolling(window=10).mean()

# 10 günlük EMA ekleme
df['EMA_10'] = df['Price'].ewm(span=10, adjust=False).mean()
df.head()

,Date,Price,Open,High,Low,Change %,SMA_10,EMA_10
0,2016-09-08,2.736,2.702,2.754,2.702,-0.29,NaN,2.736000
1,2016-09-07,2.744,2.755,2.766,2.730,0.18,NaN,2.737455
2,2016-09-06,2.739,2.739,2.748,2.725,-0.22,NaN,2.737736
3,2016-09-05,2.745,2.801,2.801,2.745,-0.80,NaN,2.739056
4,2016-09-02,2.767,2.770,2.782,2.734,0.62,NaN,2.744137


In [18]:
# 10 günlük SMA ekleme
df['SMA_10'] = df['Price'].rolling(window=10).mean()

# 10 günlük EMA ekleme
df['EMA_10'] = df['Price'].ewm(span=10, adjust=False).mean()
df.head()

,Date,Price,Open,High,Low,Change %,SMA_10,EMA_10
0,2016-09-08,2.736,2.702,2.754,2.702,-0.29,NaN,2.736000
1,2016-09-07,2.744,2.755,2.766,2.730,0.18,NaN,2.737455
2,2016-09-06,2.739,2.739,2.748,2.725,-0.22,NaN,2.737736
3,2016-09-05,2.745,2.801,2.801,2.745,-0.80,NaN,2.739056
4,2016-09-02,2.767,2.770,2.782,2.734,0.62,NaN,2.744137


In [25]:
df['SMA_10'] = df['SMA_10'].fillna(df['Price'].expanding().mean()) # ilk 10 gün ortalaması
df.head(20)

,Date,Price,Open,High,Low,Change %,SMA_10,EMA_10,RSI_14
0,2016-09-08,2.736,2.702,2.7540,2.7020,-0.29,2.736000,2.736000,NaN
1,2016-09-07,2.744,2.755,2.7660,2.7300,0.18,2.740000,2.737455,NaN
2,2016-09-06,2.739,2.739,2.7480,2.7250,-0.22,2.739667,2.737736,NaN
3,2016-09-05,2.745,2.801,2.8010,2.7450,-0.80,2.741000,2.739056,NaN
4,2016-09-02,2.767,2.770,2.7820,2.7340,0.62,2.746200,2.744137,NaN
5,2016-09-01,2.750,2.739,2.7970,2.7359,-0.18,2.746833,2.745203,NaN
6,2016-08-31,2.755,2.758,2.7840,2.7400,-1.57,2.748000,2.746984,NaN
7,2016-08-30,2.799,2.763,2.8130,2.7620,0.68,2.754375,2.756442,NaN
8,2016-08-26,2.780,2.780,2.7900,2.7640,-0.75,2.757222,2.760725,NaN
9,2016-08-25,2.801,2.783,2.8100,2.7490,-0.74,2.761600,2.768048,NaN


In [26]:
from ta.momentum import RSIIndicator

# 14 günlük RSI ekleme
rsi = RSIIndicator(close=df['Price'], window=14)
df['RSI_14'] = rsi.rsi()

In [27]:
#df['RSI_14'] = df['RSI_14'].fillna(df['RSI_14'].mean())

# İlk 30 günün RSI_14 ortalamasını hesapla (NaN hariç)
mean_first_20 = df.loc[:20, 'RSI_14'].mean()

# İlk 30 günün NaN değerlerini bu ortalama ile doldur
df.loc[:29, 'RSI_14'] = df.loc[:29, 'RSI_14'].fillna(mean_first_20)
df.head(20)

,Date,Price,Open,High,Low,Change %,SMA_10,EMA_10,RSI_14
0,2016-09-08,2.736,2.702,2.7540,2.7020,-0.29,2.736000,2.736000,65.602365
1,2016-09-07,2.744,2.755,2.7660,2.7300,0.18,2.740000,2.737455,65.602365
2,2016-09-06,2.739,2.739,2.7480,2.7250,-0.22,2.739667,2.737736,65.602365
3,2016-09-05,2.745,2.801,2.8010,2.7450,-0.80,2.741000,2.739056,65.602365
4,2016-09-02,2.767,2.770,2.7820,2.7340,0.62,2.746200,2.744137,65.602365
5,2016-09-01,2.750,2.739,2.7970,2.7359,-0.18,2.746833,2.745203,65.602365
6,2016-08-31,2.755,2.758,2.7840,2.7400,-1.57,2.748000,2.746984,65.602365
7,2016-08-30,2.799,2.763,2.8130,2.7620,0.68,2.754375,2.756442,65.602365
8,2016-08-26,2.780,2.780,2.7900,2.7640,-0.75,2.757222,2.760725,65.602365
9,2016-08-25,2.801,2.783,2.8100,2.7490,-0.74,2.761600,2.768048,65.602365


In [28]:
df.dtypes

Date        datetime64[ns]
Price              float64
Open               float64
High               float64
Low                float64
Change %            object
SMA_10             float64
EMA_10             float64
RSI_14             float64
dtype: object

In [31]:
from ta.trend import MACD

# MACD ekleme (12, 26, 9)
macd = MACD(close=df['Price'], window_slow=26, window_fast=12, window_sign=9)
df['MACD'] = macd.macd()          # MACD çizgisi
df['MACD_Signal'] = macd.macd_signal()  # Sinyal çizgisi
df['MACD_Diff'] = macd.macd_diff()      # Fark (histogram)

In [32]:
df.head(40)

,Date,Price,Open,High,Low,Change %,SMA_10,EMA_10,RSI_14,MACD,MACD_Signal,MACD_Diff
0,2016-09-08,2.736,2.702,2.7540,2.7020,-0.29,2.736000,2.736000,65.602365,NaN,NaN,NaN
1,2016-09-07,2.744,2.755,2.7660,2.7300,0.18,2.740000,2.737455,65.602365,NaN,NaN,NaN
2,2016-09-06,2.739,2.739,2.7480,2.7250,-0.22,2.739667,2.737736,65.602365,NaN,NaN,NaN
3,2016-09-05,2.745,2.801,2.8010,2.7450,-0.80,2.741000,2.739056,65.602365,NaN,NaN,NaN
4,2016-09-02,2.767,2.770,2.7820,2.7340,0.62,2.746200,2.744137,65.602365,NaN,NaN,NaN
5,2016-09-01,2.750,2.739,2.7970,2.7359,-0.18,2.746833,2.745203,65.602365,NaN,NaN,NaN
6,2016-08-31,2.755,2.758,2.7840,2.7400,-1.57,2.748000,2.746984,65.602365,NaN,NaN,NaN
7,2016-08-30,2.799,2.763,2.8130,2.7620,0.68,2.754375,2.756442,65.602365,NaN,NaN,NaN
8,2016-08-26,2.780,2.780,2.7900,2.7640,-0.75,2.757222,2.760725,65.602365,NaN,NaN,NaN
9,2016-08-25,2.801,2.783,2.8100,2.7490,-0.74,2.761600,2.768048,65.602365,NaN,NaN,NaN


In [33]:
for col in ['MACD', 'MACD_Signal', 'MACD_Diff']: # İlk günki değer ile dolduruldu
    first_value = df[col].iloc[33]
    df[col] = df[col].fillna(first_value)
df.isnull().sum()

Date           0
Price          0
Open           0
High           0
Low            0
Change %       0
SMA_10         0
EMA_10         0
RSI_14         0
MACD           0
MACD_Signal    0
MACD_Diff      0
dtype: int64

In [34]:
df.dtypes

Date           datetime64[ns]
Price                 float64
Open                  float64
High                  float64
Low                   float64
Change %               object
SMA_10                float64
EMA_10                float64
RSI_14                float64
MACD                  float64
MACD_Signal           float64
MACD_Diff             float64
dtype: object

In [35]:
df3 = df.copy() # her ihtimale karşı verileri koruma
df3.head(40)

,Date,Price,Open,High,Low,Change %,SMA_10,EMA_10,RSI_14,MACD,MACD_Signal,MACD_Diff
0,2016-09-08,2.736,2.702,2.7540,2.7020,-0.29,2.736000,2.736000,65.602365,-0.017379,-0.011022,-0.006357
1,2016-09-07,2.744,2.755,2.7660,2.7300,0.18,2.740000,2.737455,65.602365,-0.017379,-0.011022,-0.006357
2,2016-09-06,2.739,2.739,2.7480,2.7250,-0.22,2.739667,2.737736,65.602365,-0.017379,-0.011022,-0.006357
3,2016-09-05,2.745,2.801,2.8010,2.7450,-0.80,2.741000,2.739056,65.602365,-0.017379,-0.011022,-0.006357
4,2016-09-02,2.767,2.770,2.7820,2.7340,0.62,2.746200,2.744137,65.602365,-0.017379,-0.011022,-0.006357
5,2016-09-01,2.750,2.739,2.7970,2.7359,-0.18,2.746833,2.745203,65.602365,-0.017379,-0.011022,-0.006357
6,2016-08-31,2.755,2.758,2.7840,2.7400,-1.57,2.748000,2.746984,65.602365,-0.017379,-0.011022,-0.006357
7,2016-08-30,2.799,2.763,2.8130,2.7620,0.68,2.754375,2.756442,65.602365,-0.017379,-0.011022,-0.006357
8,2016-08-26,2.780,2.780,2.7900,2.7640,-0.75,2.757222,2.760725,65.602365,-0.017379,-0.011022,-0.006357
9,2016-08-25,2.801,2.783,2.8100,2.7490,-0.74,2.761600,2.768048,65.602365,-0.017379,-0.011022,-0.006357


In [36]:
df3.to_csv('London_Stock_Exchange_without_normalization.csv',index=False)

In [37]:
from sklearn.preprocessing import StandardScaler

# Normalleştirilecek sütunlar
normalize_columns = ['Price', 'Open', 'High', 'Low', 'SMA_10', 'EMA_10', 'MACD', 'MACD_Signal', 'MACD_Diff']

# StandardScaler ile standartlaştırma
scaler = StandardScaler()
df3[normalize_columns] = scaler.fit_transform(df3[normalize_columns])

# Sonucu kontrol etme
print(df3.head())

        Date     Price      Open      High       Low Change %    SMA_10  \
0 2016-09-08 -0.813786 -0.820169 -0.791867 -0.838758    -0.29 -0.853871   
1 2016-09-07 -0.813784 -0.820156 -0.791864 -0.838751     0.18 -0.853869   
2 2016-09-06 -0.813786 -0.820160 -0.791868 -0.838752    -0.22 -0.853870   
3 2016-09-05 -0.813784 -0.820145 -0.791855 -0.838747    -0.80 -0.853869   
4 2016-09-02 -0.813779 -0.820153 -0.791860 -0.838750     0.62 -0.853868   

     EMA_10     RSI_14      MACD  MACD_Signal  MACD_Diff  
0 -0.862547  65.602365 -0.038403    -0.040481  -0.007558  
1 -0.862547  65.602365 -0.038403    -0.040481  -0.007558  
2 -0.862546  65.602365 -0.038403    -0.040481  -0.007558  
3 -0.862546  65.602365 -0.038403    -0.040481  -0.007558  
4 -0.862545  65.602365 -0.038403    -0.040481  -0.007558  


In [38]:
df3.to_csv("London_Stock_Exchange_normalization.csv",index=False)